# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `/data/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [52]:
import pandas as pd

file_path = 'complications_all.csv'
all_hospitals = pd.read_csv(file_path)

# This display the first few rows and general information about the dataset for general idea how it looks like
all_hospitals.head(), all_hospitals.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91395 entries, 0 to 91394
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Facility ID           91395 non-null  object 
 1   Facility Name         91395 non-null  object 
 2   Address               91395 non-null  object 
 3   City                  91395 non-null  object 
 4   State                 91395 non-null  object 
 5   ZIP Code              91395 non-null  int64  
 6   County Name           91395 non-null  object 
 7   Phone Number          91395 non-null  object 
 8   Measure ID            91395 non-null  object 
 9   Measure Name          91395 non-null  object 
 10  Compared to National  91395 non-null  object 
 11  Denominator           91395 non-null  object 
 12  Score                 91395 non-null  object 
 13  Lower Estimate        91395 non-null  object 
 14  Higher Estimate       91395 non-null  object 
 15  Footnote           

(  Facility ID                     Facility Name                 Address  \
 0      010001  SOUTHEAST ALABAMA MEDICAL CENTER  1108 ROSS CLARK CIRCLE   
 1      010001  SOUTHEAST ALABAMA MEDICAL CENTER  1108 ROSS CLARK CIRCLE   
 2      010001  SOUTHEAST ALABAMA MEDICAL CENTER  1108 ROSS CLARK CIRCLE   
 3      010001  SOUTHEAST ALABAMA MEDICAL CENTER  1108 ROSS CLARK CIRCLE   
 4      010001  SOUTHEAST ALABAMA MEDICAL CENTER  1108 ROSS CLARK CIRCLE   
 
      City State  ZIP Code County Name    Phone Number     Measure ID  \
 0  DOTHAN    AL     36301     HOUSTON  (334) 793-8701  COMP_HIP_KNEE   
 1  DOTHAN    AL     36301     HOUSTON  (334) 793-8701    MORT_30_AMI   
 2  DOTHAN    AL     36301     HOUSTON  (334) 793-8701   MORT_30_CABG   
 3  DOTHAN    AL     36301     HOUSTON  (334) 793-8701   MORT_30_COPD   
 4  DOTHAN    AL     36301     HOUSTON  (334) 793-8701     MORT_30_HF   
 
                                         Measure Name  \
 0  Rate of complications for hip/knee replac

In [53]:
def load_and_filter_data(file_path):
    """
    This function loads the CSV data and filters only hospitals in Missouri (MO).

    Args:
    - file_path (str): This is where the CSV file is located.

    Returns:
    - pd.DataFrame: This is the DataFrame with only Missouri hospitals.

    Example:
    >>> df = load_and_filter_data('/mnt/data/complications_all.csv')
    >>> df['State'].nunique()
    1
    >>> df['State'].unique()
    array(['MO'], dtype=object)
    """
    # First we need to read the CSV file with data of all hospitals
    all_hospitals = pd.read_csv(file_path)

    # Now we need to keep only rows where the hospital is in Missouri (MO) as per the instructions
    mo_hospitals = all_hospitals[all_hospitals['State'] == 'MO']

    return mo_hospitals


In [54]:
def clean_and_convert_columns(mo_hospitals):
    """
    This function changes the 'Start Date' and 'End Date' columns to date format.
    It also removes rows where the 'Denominator' is 'Not Available' and makes it numeric.

    Args:
    - mo_hospitals (pd.DataFrame): Data only for Missouri hospitals.

    Returns:
    - pd.DataFrame: Data with cleaned 'Start Date' and 'End Date' as date type
                    and 'Denominator' as a number.

    Example:
    >>> data = pd.DataFrame({
    ...     'Start Date': ['2017-01-01', 'Not Available'],
    ...     'End Date': ['2018-01-01', '2019-01-01'],
    ...     'Denominator': ['100', 'Not Available']
    ... })
    >>> clean_data = clean_and_convert_columns(data)
    >>> clean_data['Start Date'].dtype == 'datetime64[ns]'
    True
    >>> clean_data['Denominator'].dtype == 'int64'
    True
    >>> clean_data.shape[0]  # Only rows with numeric 'Denominator' remain
    1
    """
    # In this step we will convert 'Start Date' and 'End Date' to date type
    mo_hospitals['Start Date'] = pd.to_datetime(mo_hospitals['Start Date'], errors='coerce')
    mo_hospitals['End Date'] = pd.to_datetime(mo_hospitals['End Date'], errors='coerce')

    # Next we have to remove rows where 'Denominator' is 'Not Available'
    mo_hospitals = mo_hospitals[mo_hospitals['Denominator'] != 'Not Available']

    # Finally, we have to make 'Denominator' column numeric to ease out the process
    mo_hospitals['Denominator'] = pd.to_numeric(mo_hospitals['Denominator'])

    return mo_hospitals


In [55]:
def aggregate_data(mo_hospitals):
    """
    This function groups data by hospital and calculates the minimum start date,
    maximum end date, and total denominator for each hospital.

    Args:
    - mo_hospitals (pd.DataFrame): Cleaned data of Missouri hospitals.

    Returns:
    - pd.DataFrame: Data with each hospital’s earliest start, latest end date, and total denominator.

    Example:
    >>> data = pd.DataFrame({
    ...     'Facility Name': ['A', 'A', 'B'],
    ...     'Start Date': ['2015-04-01', '2016-01-01', '2017-01-01'],
    ...     'End Date': ['2018-06-30', '2018-01-01', '2018-06-30'],
    ...     'Denominator': [500, 300, 200]
    ... })
    >>> data['Start Date'] = pd.to_datetime(data['Start Date'])
    >>> data['End Date'] = pd.to_datetime(data['End Date'])
    >>> mo_summary = aggregate_data(data)
    >>> mo_summary.loc['A', 'number']
    800
    >>> mo_summary.loc['A', 'start_date'] == pd.Timestamp('2015-04-01')
    True
    >>> mo_summary.loc['B', 'end_date'] == pd.Timestamp('2018-06-30')
    True
    """
    # Now we need to group by each hospital and find the minimum start date, maximum end date, and sum the denominators
    mo_summary = mo_hospitals.groupby('Facility Name').agg(
        start_date=('Start Date', 'min'),
        end_date=('End Date', 'max'),
        number=('Denominator', 'sum')
    )

    return mo_summary


In [56]:
# Process the data step-by-step to have clear result in the final visuvalization of it
mo_hospitals = load_and_filter_data(file_path)
mo_hospitals = clean_and_convert_columns(mo_hospitals)
mo_summary = aggregate_data(mo_hospitals)

<ipython-input-54-9bfa68420523>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['Denominator'] = pd.to_numeric(mo_hospitals['Denominator'])


In [57]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

In [59]:
mo_summary.head() # This prints the data for visualization and also give interactive sheets for easy analysis and better understanding

,start_date,end_date,number
Facility Name,,,
BARNES JEWISH HOSPITAL,2015-04-01,2018-06-30,131313
BARNES-JEWISH ST PETERS HOSPITAL,2015-04-01,2018-06-30,15668
BARNES-JEWISH WEST COUNTY HOSPITAL,2015-04-01,2018-06-30,9622
BATES COUNTY MEMORIAL HOSPITAL,2015-07-01,2018-06-30,3117
BELTON REGIONAL MEDICAL CENTER,2015-04-01,2018-06-30,9270


---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

# For my project, I plan to use data from the following sources:


**1) AWS S3**: I will access publicly available datasets on AWS S3. For instance, the **Amazon Public Datasets**(https://aws.amazon.com/blogs/publicsector/aws-public-datasets-unlocking-the-potential-of-open-data-in-the-cloud/) can provide various health-related datasets.


**2) Relational Databases**: I can utilize sample databases available for educational purposes, such as the **Chinook Database**(https://github.com/lerocha/chinook-database) or **Sakila Database**(https://dev.mysql.com/doc/sakila/en/), which give direct link that can be opened in github.



**3) Web Services**: I will explore APIs that provide health data, such as the **HealthAPI** (https://www.healthdata.gov/). This site offers APIs that aggregate health statistics and data.

**4) Local Files**: I can download CSV or Excel files containing healthcare statistics or studies, such as datasets from **Kaggle** (https://www.kaggle.com/datasets)that often have various health-related data.



#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

As per the instructions from above, my project should involve data that comes in different file formats. I plan to incorporate the following formats:

**1) CSV**: Many datasets, like those found on Kaggle,(https://www.kaggle.com/datasets) come in CSV format. For example, the World Health Organization's Global Health Observatory offers various health datasets in CSV format(https://www.who.int/data/gho).


**2) JSON**: I can use APIs like the OpenFDA to obtain data in JSON format.
For example this link https://github.com/FDA/openfda/blob/master/api/faers/test/integration/cors_test.js has the raw file in the format of JSON from the github which was originally obtained from https://open.fda.gov/apis/downloads/originally  

**3)Excel**: Datasets from sources like Data.gov (https://catalog.data.gov/dataset/) often come in Excel format, such as health department reports.


**4)XML**: Some health-related data feeds may be available in XML format, which I can find through sources like the National Library of Medicine (https://clinicaltrials.gov/data-api/api).

#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

The purpose of my project is to develop a comprehensive data analysis tool that can visualize and analyze health data trends over time, which I have learnt to some extent in this HDS  class coupled with the knowledge that I have obtained from my Data Management Class that I have been taking online, for gathering datasets and performing basic SQL operations using python and R language. Apart from this, it also provides insights that could assist healthcare professionals in decision-making and laying out protocols for Standard Operating Procedures (SOP's), Which can greatly reduced the time for Pre and Post OP'S in medical field.


By integrating multiple data sources, I aim to create a unified platform that can process and present health information effectively, demonstrating the importance of data-driven approaches in improving healthcare outcomes. This project will not only showcase my technical skills but also highlight the potential impact of data analysis in a real-world healthcare setting.

P.S: I have provided the link of the datasets from the website and the direct page of the url for accessing these datasets, in the above. So that even I can also have an easy access while actually do the Final Project.



---



## Submit your work via GitHub as normal
